Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.


# Lab 3: ES

## Task

Write agents able to play [_Nim_](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., _subtraction game_).

The goal of the game is to **avoid** taking the last object.

- Task2.1: An agent using fixed rules based on _nim-sum_ (i.e., an _expert system_)
- Task2.2: An agent using evolved rules using ES

## Instructions

- Create the directory `lab2` inside the course repo
- Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

- Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
- [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.


In [1]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy


## The _Nim_ and _Nimply_ classes


In [2]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [3]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


- Class `Nim` is defined, with an initializer method `__init__`, three methods `__bool__`, `__str__`, and `nimming`, and a property `rows`.
- The `__init__` method takes two arguments: `num_rows` and `k` (with a default value of `None`), and initializes `self._rows` with a list of odd numbers and stores `k` in `self._k`.
- The `__bool__` method returns a boolean value indicating whether the sum of the numbers in `self._rows` is greater than 0.
- The `__str__` method returns a string representation of the `Nim` instance by joining the numbers in `self._rows` into a string enclosed in angle brackets.
- The `rows` property returns a tuple containing the numbers in `self._rows`.
- The `nimming` method takes a `Nimply` instance (or a tuple), unpacks it into `row` and `num_objects`, and performs two assertions. It subtracts `num_objects` from the number of objects in the specified row if both assertions pass.


## Sample (and silly) startegies


In [4]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0]) # select random non empty row
    num_objects = random.randint(1, state.rows[row]) # select random number of objects to be removed from row
    return Nimply(row, num_objects) #return new Nimply object for that row with updated number of objects


In [5]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)] #create list of all possible moves
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1]))) #return Nimply object with maximum number of objects from lowest row


In [7]:
def adaptive1(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}  # set initial value for love_small

    if state.rows[0] <= 3:  # if lowest row has 3 or less objects
        genome["love_small"] = 0.9  # increase love_small
    elif state.rows[0] >= 7:  # if lowest row has 7 or more objects
        genome["love_small"] = 0.1  # decrease love_small

    row = min(
        range(len(state.rows)), key=lambda r: state.rows[r]
    )  # select row with lowest number of objects

    num_objects = int(
        genome["love_small"] * state.rows[row]
    )  # select number of objects to be removed from row
    
    return Nimply(
        row, num_objects
    )  # return Nimply object for that row with updated number of objects

In [8]:
import numpy as np


def nim_sum(state: Nim) -> int:
    """
    takes a `Nim` object and calculates the Nim-sum of the current game
    state by converting the counts of objects in the rows to binary, summing
    them, and taking the modulo 2 to obtain an integer Nim-sum
    """
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    """
    takes a `Nim` object and returns a dictionary with a single key,
    "possible_moves," which maps each possible move to its resulting Nim-sum. To
    calculate this, it iterates over all possible moves, makes a deep copy of
    the game state, applies the move to the copy, and calculates the Nim-sum of
    the resulting state
    """
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    """
    takes a `Nim` object and returns an optimal move by analyzing the
    current game state to get the Nim-sums of all possible moves. It selects
    moves that result in a non-zero Nim-sum, representing "winning" moves.
    If no such moves exist, it selects all moves and chooses one at random
    to return as the optimal move
    """
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply

## Oversimplified match

- The line `logging.getLogger().setLevel(logging.INFO)` sets the logging level to `INFO`, indicating that all logging messages with level `INFO` or higher will be displayed.

- The `strategy` tuple contains two functions, `optimal` and `pure_random`, which are used to determine the moves of the two players.

- The `while nim:` loop continues as long as the game is not over, meaning there are objects remaining in the game. Inside the loop, the current player's strategy function is invoked with the current game state to determine the next move (`ply = strategy[player](nim)`). This move is then applied to the game (`nim.nimming(ply)`), and both the game state and the move are logged. Finally, the current player is switched using `player = 1 - player`.

- After the loop, the final game state is logged, along with the player who won the game.


In [9]:
logging.getLogger().setLevel(logging.INFO)

# strategy = (optimal, pure_random)
# strategy = (gabriele, adaptive1)
strategy = (optimal, adaptive1)

nim = Nim(5)
logging.info(f"init : {nim}\n")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}\n")
    player = 1 - player
logging.info(f"status: Player {player} won using {strategy[player].__name__} startegy!")

INFO:root:init : <1 3 5 7 9>

INFO:root:ply: player 0 plays Nimply(row=3, num_objects=6)
INFO:root:status: <1 3 5 1 9>

INFO:root:ply: player 1 plays Nimply(row=0, num_objects=0)
INFO:root:status: <1 3 5 1 9>

INFO:root:ply: player 0 plays Nimply(row=4, num_objects=5)
INFO:root:status: <1 3 5 1 4>

INFO:root:ply: player 1 plays Nimply(row=0, num_objects=0)
INFO:root:status: <1 3 5 1 4>

INFO:root:ply: player 0 plays Nimply(row=2, num_objects=3)
INFO:root:status: <1 3 2 1 4>

INFO:root:ply: player 1 plays Nimply(row=0, num_objects=0)
INFO:root:status: <1 3 2 1 4>

INFO:root:ply: player 0 plays Nimply(row=4, num_objects=1)
INFO:root:status: <1 3 2 1 3>

INFO:root:ply: player 1 plays Nimply(row=0, num_objects=0)
INFO:root:status: <1 3 2 1 3>

INFO:root:ply: player 0 plays Nimply(row=4, num_objects=1)
INFO:root:status: <1 3 2 1 2>

INFO:root:ply: player 1 plays Nimply(row=0, num_objects=0)
INFO:root:status: <1 3 2 1 2>

INFO:root:ply: player 0 plays Nimply(row=4, num_objects=2)
INFO:root:s